In [1]:
import os
import json
import itertools
from domain_groups import group_domains
from collections import defaultdict, Counter
DIR = "../optimization/"

DOMAINS = [
    "barman", "blocksworld", "childsnack", "data-network", "depots", "driverlog",
    "elevators", "floortile", "grid", "gripper", "hiking", "logistics",
    "miconic", "nomystery", "openstacks", "parking", "rovers", "satellite",
    "scanalyzer", "snake", "storage", "tpp", "transport", "visitall",
    "woodworking", "zenotravel",
]

TRAINING_PLANNERS_OPT = [
    "fd1906-blind",
    "fd1906-fdss1-mas1-60s",
    "fd1906-fdss1-mas2-60s",
    "fd1906-bjolp",
    "fd1906-lmcut",
    "ipc2014-opt-symba1",
]
EVALUATION_PLANNERS_OPT = [
    "ipc2018-opt-complementary2-3584mb",
    "ipc2018-opt-decstar",
    "ipc2018-opt-delfi-blind",
    "ipc2018-opt-delfi-celmcut",
    "ipc2018-opt-delfi-ipdb",
    "ipc2018-opt-delfi-mas-miasm",
    "ipc2018-opt-delfi-mas-sccdfp",
    "ipc2018-opt-scorpion",
]

TRAINING_PLANNERS_SAT = [
    "fd1906-gbfs-ff",
    "fd1906-lama-first",
    "ipc2014-agl-mpc",
    "ipc2014-agl-mercury",
    "ipc2014-agl-jasper",
    "ipc2014-agl-probe",
]
EVALUATION_PLANNERS_SAT = [
    "ipc2018-agl-decstar",
    "ipc2018-agl-fd-remix",
    "ipc2018-agl-lapkt-dual-bfws",
    "ipc2018-agl-lapkt-bfws-pref",
    "ipc2018-agl-lapkt-poly-bfws",
    "ipc2018-agl-olcff",
    "ipc2018-agl-saarplan",
    "ipc2018-agl-cerberus",
]

TRAINING_PLANNERS = {
        "opt-ipc" : TRAINING_PLANNERS_OPT,
        "opt-1204": TRAINING_PLANNERS_OPT,
        "opt-1210": TRAINING_PLANNERS_OPT,
        "sat-ipc": TRAINING_PLANNERS_SAT,
        "sat-1204": TRAINING_PLANNERS_SAT,
        "sat-1210": TRAINING_PLANNERS_SAT,
}

EVALUATION_PLANNERS = {
        "opt-ipc" : EVALUATION_PLANNERS_OPT,
        "opt-1204": EVALUATION_PLANNERS_OPT,
        "opt-1210": EVALUATION_PLANNERS_OPT,
        "sat-ipc": EVALUATION_PLANNERS_SAT,
        "sat-1204": EVALUATION_PLANNERS_SAT,
        "sat-1210": EVALUATION_PLANNERS_SAT,
}


FILENAMES = {
        "opt-ipc": ["2020-11-23-A-optimization-planners-ipc-properties.json", "2020-12-05-A-evaluation-opt-ipc-properties.json"],
        "sat-ipc": ["2020-11-23-D-optimization-planners-sat-ipc-properties.json","2020-12-05-B-evaluation-sat-ipc-properties.json"],
        "opt-1210": ["2020-12-13-A-evaluation-opt-new2014-properties.json"],
        "sat-1210": ["2020-12-13-B-evaluation-sat-new2014-properties.json"],

    }

def read_runs(filename):
    with open(filename) as f:
        data = json.load(f)
        for run, values in data.items(): 
            if "coverage" not in values: 
                #print ("Warning, coverage attribute missing in ", filename, run)
                pass
        return data.values()

# Step 1: gather all runs and label them by dataset and domain
all_runs = defaultdict(list)
for name, filenames in FILENAMES.items():
    
    for filename in filenames: 
        new_runs = read_runs(os.path.join(DIR, "results", filename))
        new_runs = [group_domains(run) for run in new_runs]
        new_runs = [run for run in new_runs if run["domain"] in DOMAINS]
        
        for run in new_runs: 
            run["dataset"] = name
            all_runs[(name, run["domain"])].append(run)
        
        

In [2]:
def compute_coverage (properties, atr_name, runs, time_limit=100000):
    coverage = Counter()
    for run in runs: 
        if "coverage" in run and run["coverage"] == 1 and run["planner_wall_clock_time"] <= time_limit: 
            coverage[run["algorithm"]] += 1
    properties[atr_name] = coverage
    

def compute_comparisons (properties, comp_atr_name, cov_atr_name, planners):
    coverage = properties[cov_atr_name]
    properties[comp_atr_name] = sum ([1 for (p1, p2) in itertools.combinations(planners, 2) if coverage[p1] != coverage[p2]])
    
    
def compute_runtimes (properties, atr_name, runs, planners):
    minimum_runtime = {}
    for run in runs: 
        if "coverage" in run and run["coverage"] == 1 and run["algorithm"] in planners: 
            problem_id = int(run["problem"].split("-p")[1].replace(".pddl", ""))
            if problem_id in minimum_runtime: 
                minimum_runtime[problem_id]= min(minimum_runtime[problem_id], run["planner_wall_clock_time"])
            else: 
                minimum_runtime[problem_id] = run["planner_wall_clock_time"]
                
    properties[atr_name] = [minimum_runtime[p] if p in minimum_runtime else "unsolved" for p in range(1, 31)]

    

# Step 2: Compute all aggregated properties for a dataset
properties_dataset = defaultdict(dict)
for dataset, domain in all_runs: 
    if "-ipc" in dataset: 
        continue
    ipcdataset = dataset.split("-")[0] + "-ipc"
    
    properties_dataset[(dataset, domain)] ["track"] = dataset
    properties_dataset[(dataset, domain)] ["domain"] = domain
    compute_coverage(properties_dataset[(dataset, domain)], "coverage", all_runs[(dataset, domain)])
    compute_coverage(properties_dataset[(dataset, domain)], "coverage30s", all_runs[(dataset, domain)], 30)
    compute_coverage(properties_dataset[(dataset, domain)], "coverage300s", all_runs[(dataset, domain)], 300)
        
    compute_runtimes(properties_dataset[(dataset, domain)], "runtimes-training", all_runs[(dataset, domain)], TRAINING_PLANNERS[dataset])
    compute_runtimes(properties_dataset[(dataset, domain)], "runtimes-eval", all_runs[(dataset, domain)], EVALUATION_PLANNERS[dataset])

    compute_coverage(properties_dataset[(dataset, domain)], "coverage-ipc", all_runs[(ipcdataset, domain)])
    compute_coverage(properties_dataset[(dataset, domain)], "coverage30s-ipc", all_runs[(ipcdataset, domain)], 30)
    compute_coverage(properties_dataset[(dataset, domain)], "coverage300s-ipc", all_runs[(ipcdataset, domain)], 300)

    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons-training", "coverage", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons-eval", "coverage", EVALUATION_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons30s-training", "coverage30s", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons30s-eval", "coverage30s", EVALUATION_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons300s-training", "coverage300s", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons300s-eval", "coverage300s", EVALUATION_PLANNERS[dataset])

    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons-training-ipc", "coverage-ipc", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons-eval-ipc", "coverage-ipc", EVALUATION_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons30s-training-ipc", "coverage30s-ipc", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons30s-eval-ipc", "coverage30s-ipc", EVALUATION_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons300s-training-ipc", "coverage300s-ipc", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons300s-eval-ipc", "coverage300s-ipc", EVALUATION_PLANNERS[dataset])
    
    for x in ["comparisons-training", "comparisons-eval", "comparisons300s-training", "comparisons300s-eval" ]:
        properties_dataset[(dataset, domain)][f"{x}-ipcdiff"] = properties_dataset[(dataset, domain)][x] - properties_dataset[(dataset, domain)][f"{x}-ipc"] 
    


In [3]:
LOGDIRS = {"1204" : "../logfiles/2020-12-04/", 
           "1210" : "../logfiles/2020-12-10/"
          }
import os
import re

def parse_CPLEX_log(content):
    
    regex_int = [re.compile(x) for x in [".*Different evaluated sequences: (?P<evaluated_sequences>(\d+))",
                                     ".*Candidate sequences: (?P<candidate_sequences>(\d+))", 
                                    ]]
    regex_float = [re.compile(x) for x in [".*Solution value  = (?P<solution_value>(.*))",]]

    penalties = [float(l.split(":")[-1]) for l in content if l.startswith("Penalty: ")]
        
    data = {
            "sart_runtimes" : False, 
            "num_sequences" : 0, 
            "min_penalty" : min(penalties) if penalties else "-", 
            "max_penalty" : max(penalties) if penalties else "-",  
            "runtimes-estimated" : []
           }
    for l in content: 
        if "Using sart runtimes on CPLEX optimization" in l: 
            data["sart_runtimes"] = True
            continue
            
        if "Selected: sequence" in l:
            from_instance, to_instance = list(map(int, l.split("from")[1].split("to")))
            data["num_sequences"] += 1
            
        if l.startswith("Estimated runtimes:"):             
            runtimes_sequence = list(map(float, l.split(":")[1].split(",")))
            if len(runtimes_sequence) == 30:
                data["runtimes-estimated"] += runtimes_sequence[from_instance:to_instance+1]
            else:
                data["runtimes-estimated"] += runtimes_sequence[:to_instance-from_instance+1]
            

            
        for reg in regex_int: 
            if reg.match(l): 
                for atr, value in reg.match(l).groupdict().items():
                    data [atr] = int(value)
        
        for reg in regex_float: 
            if reg.match(l): 
                for atr, value in reg.match(l).groupdict().items():
                    data [atr] = float(value)
                    
    assert len(data["runtimes-estimated"]) == 30, (data["runtimes-estimated"], len(data["runtimes-estimated"]))
    return data

for dset, logdir in LOGDIRS.items(): 
    for logfile in sorted(os.listdir(logdir)):
        _, track, domain = logfile.split("_")
    
        dataset = track.replace("14", f"-{dset}")
    
        f = open(f"{logdir}/{logfile}")
        content = f.readlines()
    
        properties_dataset[(dataset, domain)] ["track"] = dataset
        properties_dataset[(dataset, domain)] ["domain"] = domain

        data = parse_CPLEX_log(content)
        properties_dataset[(dataset, domain)].update(data)
    



                 

In [4]:
import statistics
def compute_smoothness(properties, atr_name, runtimes):
    runtimes = sorted([r for r in runtimes if r != "unsolved"])
    
    factors = [r/runtimes[i] for i, r in enumerate (runtimes[1:])]
    runtimes_g5 = [r for r in runtimes if r > 5]
    factors_g5 = [r/runtimes_g5[i] for i, r in enumerate (runtimes_g5[1:])]
    
    properties [f"max_runtime_{atr_name}"] = max(runtimes) if runtimes else "-"
    properties [f"min_runtime_{atr_name}"] = min(runtimes) if runtimes else "-"
    
    properties [f"min_factor_runtime_{atr_name}"] = min(factors) if factors else "-"
    properties [f"max_factor_runtime_{atr_name}"] = max(factors) if factors else "-"
    properties [f"gmean_factor_runtime_{atr_name}"] = statistics.geometric_mean(factors) if factors else "-"
    
    properties [f"min_factorg5_runtime_{atr_name}"] = min(factors_g5) if factors_g5 else "-"
    properties [f"max_factorg5_runtime_{atr_name}"] = max(factors_g5) if factors_g5 else "-" 
    properties [f"gmean_factorg5_runtime_{atr_name}"] = statistics.geometric_mean(factors_g5) if factors_g5 else "-"
    
    return properties

for dataset, domain in properties_dataset: 
    properties_dataset[(dataset, domain)] = compute_smoothness(properties_dataset[(dataset, domain)], "estimated", properties_dataset[(dataset, domain)]["runtimes-estimated"])
    properties_dataset[(dataset, domain)] = compute_smoothness(properties_dataset[(dataset, domain)], "training", properties_dataset[(dataset, domain)]["runtimes-training"])
    properties_dataset[(dataset, domain)] = compute_smoothness(properties_dataset[(dataset, domain)], "eval", properties_dataset[(dataset, domain)]["runtimes-eval"])
        

KeyError: 'runtimes-training'

In [24]:
import statistics

def estimated_error(properties):     
    properties["coverage-estimated"] = len([r   for r in properties["runtimes-estimated"] if r < 1800])
    properties["coverage-training"] = len([r  for r in properties["runtimes-training"] if r != "unsolved"])
    properties["estimated-error-coverage"] = abs(properties["coverage-estimated"] - properties["coverage-training"])

    properties["estimated-instance-error-coverage"] = 0
    properties["estimated-instance-error-runtime"] = 0
    paired_runtimes = zip([r if r < 1800 else "unsolved" for r in properties["runtimes-estimated"]], properties["runtimes-training"])
    for (est, tr) in paired_runtimes: 
        if est == tr: 
            continue
        if est == "unsolved" or tr == "unsolved":     
            properties["estimated-instance-error-coverage"] += 1
        else: 
            properties["estimated-instance-error-runtime"] += abs(est - tr)

    paired_sorted_runtimes = zip([r for r in sorted(properties["runtimes-estimated"]) if r < 1800], sorted([r for r in properties["runtimes-training"] if r != "unsolved"]))
    
    error_runtime = [max(est,tr)/min(est, tr) for (est, tr) in paired_sorted_runtimes]
    
    properties["estimated-error-runtime"] = statistics.geometric_mean(error_runtime) if error_runtime else "a lot"
        

for dataset, domain in properties_dataset: 
    estimated_error(properties_dataset[(dataset, domain)])


In [25]:
def penalty_by_factor(factor):
    if factor <= 1:  # Runtime is not increasing: maximum penalty of 1
        return 1
    elif factor <= 1.5:
        return 3 - 2*factor
    elif factor <= 2: # Runtime is increasing, but not very quickly
        return 0
    elif factor > 2: # Runtime is increasing too quickly
        return 1 - (2 / factor)

def evaluate_runtimes_single_sequence(runtimes, num_expected_runtimes = 5):
    penalty = 0
    sorted_runtimes = sorted(runtimes)

    # The default scaling only works if all instances are solvable. For each unsolvable
    # instance apply a double penalty.
    if len(runtimes) < num_expected_runtimes:
        penalty += 2 * (num_expected_runtimes - len(runtimes))

    for i in range(1, len(runtimes)):
        factor = sorted_runtimes[i] / sorted_runtimes[i - 1]
        penalty += penalty_by_factor(factor)

    return penalty


def compute_runtime_penalty(properties, atr_name, runtimes): 
    s_runtimes = sorted([t for t in runtimes if t != "unsolved" and t >= 5]) [:5]
    penalty = evaluate_runtimes_single_sequence(s_runtimes)

    properties [f"{atr_name}"] = penalty

for dataset, domain in properties_dataset: 
    compute_runtime_penalty(properties_dataset[(dataset, domain)], "penalty-estimated", properties_dataset[(dataset, domain)]["runtimes-estimated"])
    compute_runtime_penalty(properties_dataset[(dataset, domain)], "penalty-training", properties_dataset[(dataset, domain)]["runtimes-training"])
    compute_runtime_penalty(properties_dataset[(dataset, domain)], "penalty-eval", properties_dataset[(dataset, domain)]["runtimes-eval"])


In [26]:
import pandas as pd

from IPython.display import display

def make_short(x): 
    return x.replace("comparisons", "cmp").replace("training", "tr")
ATRIBUTES_TABLE = ["track", "domain", "solution_value", "sart_runtimes", "comparisons-training", "comparisons-training-ipcdiff", "penalty-estimated", "penalty-training",  "penalty-eval",  "comparisons-eval", "comparisons-eval-ipcdiff", "estimated-error-coverage", "estimated-error-runtime", "coverage-estimated", "coverage-training"]

table = []
for data in sorted(properties_dataset.values(), key = lambda x : x["comparisons-eval-ipcdiff"] + x["comparisons-training-ipcdiff"]): 
    table.append([data[atr] if atr in data else "-" for atr in ATRIBUTES_TABLE ])

pd.options.display.float_format = "{:,.2g}".format
pd.set_option('display.max_rows', 500)

df = pd.DataFrame(table,columns =  list(map(make_short, ATRIBUTES_TABLE)))
display(df)

            

,track,domain,solution_value,sart_runtimes,cmp-tr,cmp-tr-ipcdiff,penalty-estimated,penalty-tr,penalty-eval,cmp-eval,cmp-eval-ipcdiff,estimated-error-coverage,estimated-error-runtime,coverage-estimated,coverage-tr
0,sat-1210,openstacks,12,True,0,-14,0.032,10,10,0,-21,19,a lot,19,0
1,opt-1210,openstacks,4.7,True,0,-12,0.68,10,10,0,-22,19,a lot,19,0
2,sat-1204,openstacks,69,False,5,-9,1.1,10,10,0,-21,12,3.6e+02,18,30
3,opt-1210,transport,10,True,9,-5,1.2,1.4,2.3,13,-8,4,1.5,20,24
4,opt-1204,rovers,14,True,11,-2,1.5,2.3,3.1,18,-8,3,3.9,20,23
5,opt-1204,tpp,16,True,5,-6,1,2.3,1.8,22,-2,4,1.7,20,16
6,sat-1204,logistics,8.2,True,9,-3,2.5,2.5,1.9,13,-4,7,1.1,22,15
7,sat-1204,snake,4.4,True,9,-3,0.33,0,1.8,25,-2,0,1.4,16,16
8,opt-1210,elevators,39,True,13,-2,1.8,1.3,0.67,23,-3,4,1.4,19,15
9,opt-1204,elevators,46,True,14,-1,2.2,1.9,0.55,23,-3,2,1.5,20,18


In [28]:
import pandas as pd

from IPython.display import display

def make_short(x): 
    return x.replace("comparisons", "cmp").replace("training", "tr")
ATRIBUTES_TABLE = ["track", "domain","runtimes-estimated", "runtimes-training", "runtimes-eval"]

table = []
for data in sorted(properties_dataset.values(), key = lambda x : x["comparisons-eval-ipcdiff"] + x["comparisons-training-ipcdiff"]): 
    table.append([data[atr] if atr in data else "-" for atr in ATRIBUTES_TABLE ])

pd.options.display.float_format = "{:,.2g}".format
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100000)

df = pd.DataFrame(table,columns =  list(map(make_short, ATRIBUTES_TABLE)))
display(df)


,track,domain,runtimes-estimated,runtimes-tr,runtimes-eval
0,opt-1210,transport,"[0.95, 1.0, 1.2, 1.4, 2.1, 2.6, 4.2, 7.0, 8.7, 15.0, 25.0, 29.0, 51.0, 67.0, 130.0, 210.0, 330.0, 520.0, 830.0, 1300.0, 2100.0, 3300.0, 5300.0, 8500.0, 13000.0, 21000.0, 34000.0, 54000.0, 86000.0, 140000.0]","[0.51, 0.61, 1.15, 1.32, 1.81, 2.75, 4.92, 4.99, 9.77, 9.32, 13.66, 41.09, 68.21, 93.66, 70.87, 116.76, 244.6, 265.38, 360.47, 307.8, 672.25, 1244.36, unsolved, 390.15, 892.28, unsolved, unsolved, unsolved, unsolved, unsolved]","[0.57, 0.55, 0.78, 0.72, 0.93, 1.4, 1.59, 1.34, 1.63, 2.29, 5.83, 6.2, 5.18, 12.07, 8.49, 10.85, 13.07, 13.13, 59.51, 21.45, 51.91, 84.48, 89.3, 19.22, 65.19, 97.07, 132.91, 96.94, 342.63, 220.82]"
1,opt-1204,transport,"[280.0, 410.0, 590.0, 850.0, 1200.0, 1800.0, 2600.0, 3700.0, 5300.0, 7700.0, 11000.0, 16000.0, 23000.0, 1.2, 1.3, 1.7, 2.4, 4.1, 6.5, 22.0, 41.0, 52.0, 130.0, 290.0, 660.0, 1500.0, 3400.0, 7600.0, 17000.0, 39000.0]","[156.19, 165.86, 557.85, 226.53, 1312.34, 178.66, 434.67, 697.35, 791.03, 1088.57, 321.98, 504.8, 579.05, 0.74, 1.22, 5.55, 2.54, 4.07, 6.12, 11.89, 34.46, 22.95, 54.99, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved]","[8.17, 11.97, 37.28, 18.56, 38.2, 8.49, 36.62, 35.76, 69.75, 66.47, 15.98, 17.82, 14.54, 0.6, 0.73, 1.4, 1.66, 1.36, 3.53, 6.21, 13.6, 6.31, 6.9, 538.57, unsolved, 1742.25, unsolved, 461.0, unsolved, 730.81]"
2,sat-1210,transport,"[0.6, 0.99, 2.3, 4.5, 8.7, 14.0, 21.0, 31.0, 44.0, 59.0, 78.0, 110.0, 140.0, 170.0, 240.0, 330.0, 470.0, 650.0, 910.0, 1300.0, 1800.0, 2500.0, 3500.0, 4900.0, 6800.0, 9500.0, 13000.0, 19000.0, 26000.0, 36000.0]","[0.2, 0.26, 0.57, 1.0, 3.06, 7.31, 15.25, 26.29, 39.0, 50.04, 67.92, 92.12, 117.51, 139.29, 190.42, 355.31, 322.0, 304.77, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved]","[0.43, 0.61, 1.18, 2.48, 5.5, 9.52, 18.95, 31.56, 52.14, 86.0, 75.87, 155.83, 132.51, 172.44, 732.46, 939.34, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved]"
3,sat-1204,transport,"[0.47, 0.67, 1.3, 2.4, 5.4, 8.5, 14.0, 21.0, 32.0, 41.0, 62.0, 77.0, 100.0, 130.0, 160.0, 230.0, 310.0, 440.0, 610.0, 850.0, 1200.0, 1700.0, 2300.0, 3200.0, 4500.0, 6300.0, 8800.0, 12000.0, 17000.0, 24000.0]","[0.2, 0.39, 0.52, 1.51, 4.47, 7.7, 12.37, 21.94, 32.28, 42.4, 63.04, 77.14, 102.53, 128.74, 165.57, 201.9, 254.19, 315.02, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved]","[0.39, 0.49, 0.81, 1.48, 3.63, 6.41, 12.11, 23.61, 39.25, 52.97, 76.35, 82.97, 109.1, 130.16, 304.45, 904.16, 1186.46, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved]"


In [20]:
properties_dataset["opt-1204", "tpp"]["coverage"]

Counter({'fd1906-bjolp': 2,
         'fd1906-blind': 2,
         'fd1906-fdss1-mas1-60s': 2,
         'fd1906-fdss1-mas2-60s': 2,
         'fd1906-lmcut': 2,
         'ipc2014-opt-symba1': 16,
         'ipc2018-opt-complementary2-3584mb': 15,
         'ipc2018-opt-decstar': 30,
         'ipc2018-opt-delfi-blind': 5,
         'ipc2018-opt-delfi-celmcut': 5,
         'ipc2018-opt-delfi-ipdb': 5,
         'ipc2018-opt-delfi-mas-miasm': 15,
         'ipc2018-opt-delfi-mas-sccdfp': 15,
         'ipc2018-opt-scorpion': 4})

In [21]:
properties_dataset["opt-1204", "tpp"]["coverage-ipc"]

Counter({'fd1906-bjolp': 6,
         'fd1906-blind': 6,
         'fd1906-fdss1-mas1-60s': 6,
         'fd1906-fdss1-mas2-60s': 7,
         'fd1906-lmcut': 7,
         'ipc2014-opt-symba1': 8,
         'ipc2018-opt-complementary2-3584mb': 14,
         'ipc2018-opt-decstar': 20,
         'ipc2018-opt-delfi-blind': 7,
         'ipc2018-opt-delfi-celmcut': 8,
         'ipc2018-opt-delfi-ipdb': 7,
         'ipc2018-opt-delfi-mas-miasm': 8,
         'ipc2018-opt-delfi-mas-sccdfp': 9,
         'ipc2018-opt-scorpion': 8})